In [1]:
import sys
sys.path.append('..')

In [2]:
import TechCore.Simulator.simulator_optimized as simulator
import TechCore.Strategies.Stoikov as Stoikov
import TechCore.Simulator.get_info as get_info
import TechCore.Simulator.load_data as load_data

from importlib import reload

import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm
tqdm.pandas()

path_to_file = '../TechCore/data/md/ethusdt_Binance_LinearPerpetual/'
run_time = pd.Timedelta(hours=12).delta

In [3]:
trades = pd.read_csv(path_to_file + 'trades.csv')
trades

,receive_ts,exchange_ts,aggro_side,price,size
0,1655942402218452815,1655942402217000000,BID,1049.45,0.062
1,1655942402624182869,1655942402623000000,BID,1049.45,0.726
2,1655942402624767109,1655942402623000000,BID,1049.45,0.001
3,1655942402626135583,1655942402623000000,ASK,1049.44,0.022
4,1655942402627026454,1655942402624000000,BID,1049.45,0.005
...,...,...,...,...,...
4873798,1656028780700723809,1656028780700000000,BID,1143.63,0.005
4873799,1656028780953848112,1656028780953000000,ASK,1143.62,8.744
4873800,1656028780991774028,1656028780991000000,BID,1143.63,0.066
4873801,1656028781021533873,1656028781020000000,ASK,1143.62,0.174


In [4]:
trades.index = list(trades.index)
trades

,receive_ts,exchange_ts,aggro_side,price,size
0,1655942402218452815,1655942402217000000,BID,1049.45,0.062
1,1655942402624182869,1655942402623000000,BID,1049.45,0.726
2,1655942402624767109,1655942402623000000,BID,1049.45,0.001
3,1655942402626135583,1655942402623000000,ASK,1049.44,0.022
4,1655942402627026454,1655942402624000000,BID,1049.45,0.005
...,...,...,...,...,...
4873798,1656028780700723809,1656028780700000000,BID,1143.63,0.005
4873799,1656028780953848112,1656028780953000000,ASK,1143.62,8.744
4873800,1656028780991774028,1656028780991000000,BID,1143.63,0.066
4873801,1656028781021533873,1656028781020000000,ASK,1143.62,0.174


In [5]:
trades['receive_ts'] = pd.to_datetime(trades['receive_ts'])
trades

,receive_ts,exchange_ts,aggro_side,price,size
0,2022-06-23 00:00:02.218452815,1655942402217000000,BID,1049.45,0.062
1,2022-06-23 00:00:02.624182869,1655942402623000000,BID,1049.45,0.726
2,2022-06-23 00:00:02.624767109,1655942402623000000,BID,1049.45,0.001
3,2022-06-23 00:00:02.626135583,1655942402623000000,ASK,1049.44,0.022
4,2022-06-23 00:00:02.627026454,1655942402624000000,BID,1049.45,0.005
...,...,...,...,...,...
4873798,2022-06-23 23:59:40.700723809,1656028780700000000,BID,1143.63,0.005
4873799,2022-06-23 23:59:40.953848112,1656028780953000000,ASK,1143.62,8.744
4873800,2022-06-23 23:59:40.991774028,1656028780991000000,BID,1143.63,0.066
4873801,2022-06-23 23:59:41.021533873,1656028781020000000,ASK,1143.62,0.174


In [6]:
trades['supporting: size_cumsum'] = trades['size'].cumsum()
trades

,receive_ts,exchange_ts,aggro_side,price,size,supporting: size_cumsum
0,2022-06-23 00:00:02.218452815,1655942402217000000,BID,1049.45,0.062,6.200000e-02
1,2022-06-23 00:00:02.624182869,1655942402623000000,BID,1049.45,0.726,7.880000e-01
2,2022-06-23 00:00:02.624767109,1655942402623000000,BID,1049.45,0.001,7.890000e-01
3,2022-06-23 00:00:02.626135583,1655942402623000000,ASK,1049.44,0.022,8.110000e-01
4,2022-06-23 00:00:02.627026454,1655942402624000000,BID,1049.45,0.005,8.160000e-01
...,...,...,...,...,...,...
4873798,2022-06-23 23:59:40.700723809,1656028780700000000,BID,1143.63,0.005,5.945410e+06
4873799,2022-06-23 23:59:40.953848112,1656028780953000000,ASK,1143.62,8.744,5.945419e+06
4873800,2022-06-23 23:59:40.991774028,1656028780991000000,BID,1143.63,0.066,5.945419e+06
4873801,2022-06-23 23:59:41.021533873,1656028781020000000,ASK,1143.62,0.174,5.945419e+06


In [7]:
trades['supporting: 1_sec_before'] = trades['receive_ts'].searchsorted(trades['receive_ts'] - datetime.timedelta(seconds=1))
trades

,receive_ts,exchange_ts,aggro_side,price,size,supporting: size_cumsum,supporting: 1_sec_before
0,2022-06-23 00:00:02.218452815,1655942402217000000,BID,1049.45,0.062,6.200000e-02,0
1,2022-06-23 00:00:02.624182869,1655942402623000000,BID,1049.45,0.726,7.880000e-01,0
2,2022-06-23 00:00:02.624767109,1655942402623000000,BID,1049.45,0.001,7.890000e-01,0
3,2022-06-23 00:00:02.626135583,1655942402623000000,ASK,1049.44,0.022,8.110000e-01,0
4,2022-06-23 00:00:02.627026454,1655942402624000000,BID,1049.45,0.005,8.160000e-01,0
...,...,...,...,...,...,...,...
4873798,2022-06-23 23:59:40.700723809,1656028780700000000,BID,1143.63,0.005,5.945410e+06,4873742
4873799,2022-06-23 23:59:40.953848112,1656028780953000000,ASK,1143.62,8.744,5.945419e+06,4873743
4873800,2022-06-23 23:59:40.991774028,1656028780991000000,BID,1143.63,0.066,5.945419e+06,4873743
4873801,2022-06-23 23:59:41.021533873,1656028781020000000,ASK,1143.62,0.174,5.945419e+06,4873743


In [8]:
dct = dict(zip(trades.index, trades['supporting: size_cumsum']))
trades['supporting: cs_1_sec_before'] = trades['supporting: 1_sec_before'].map(dct)
trades

,receive_ts,exchange_ts,aggro_side,price,size,supporting: size_cumsum,supporting: 1_sec_before,supporting: cs_1_sec_before
0,2022-06-23 00:00:02.218452815,1655942402217000000,BID,1049.45,0.062,6.200000e-02,0,6.200000e-02
1,2022-06-23 00:00:02.624182869,1655942402623000000,BID,1049.45,0.726,7.880000e-01,0,6.200000e-02
2,2022-06-23 00:00:02.624767109,1655942402623000000,BID,1049.45,0.001,7.890000e-01,0,6.200000e-02
3,2022-06-23 00:00:02.626135583,1655942402623000000,ASK,1049.44,0.022,8.110000e-01,0,6.200000e-02
4,2022-06-23 00:00:02.627026454,1655942402624000000,BID,1049.45,0.005,8.160000e-01,0,6.200000e-02
...,...,...,...,...,...,...,...,...
4873798,2022-06-23 23:59:40.700723809,1656028780700000000,BID,1143.63,0.005,5.945410e+06,4873742,5.945387e+06
4873799,2022-06-23 23:59:40.953848112,1656028780953000000,ASK,1143.62,8.744,5.945419e+06,4873743,5.945387e+06
4873800,2022-06-23 23:59:40.991774028,1656028780991000000,BID,1143.63,0.066,5.945419e+06,4873743,5.945387e+06
4873801,2022-06-23 23:59:41.021533873,1656028781020000000,ASK,1143.62,0.174,5.945419e+06,4873743,5.945387e+06


In [9]:
trades['order_intensity'] = trades['supporting: size_cumsum'] - trades['supporting: cs_1_sec_before']
trades

,receive_ts,exchange_ts,aggro_side,price,size,supporting: size_cumsum,supporting: 1_sec_before,supporting: cs_1_sec_before,order_intensity
0,2022-06-23 00:00:02.218452815,1655942402217000000,BID,1049.45,0.062,6.200000e-02,0,6.200000e-02,0.000
1,2022-06-23 00:00:02.624182869,1655942402623000000,BID,1049.45,0.726,7.880000e-01,0,6.200000e-02,0.726
2,2022-06-23 00:00:02.624767109,1655942402623000000,BID,1049.45,0.001,7.890000e-01,0,6.200000e-02,0.727
3,2022-06-23 00:00:02.626135583,1655942402623000000,ASK,1049.44,0.022,8.110000e-01,0,6.200000e-02,0.749
4,2022-06-23 00:00:02.627026454,1655942402624000000,BID,1049.45,0.005,8.160000e-01,0,6.200000e-02,0.754
...,...,...,...,...,...,...,...,...,...
4873798,2022-06-23 23:59:40.700723809,1656028780700000000,BID,1143.63,0.005,5.945410e+06,4873742,5.945387e+06,23.030
4873799,2022-06-23 23:59:40.953848112,1656028780953000000,ASK,1143.62,8.744,5.945419e+06,4873743,5.945387e+06,31.754
4873800,2022-06-23 23:59:40.991774028,1656028780991000000,BID,1143.63,0.066,5.945419e+06,4873743,5.945387e+06,31.820
4873801,2022-06-23 23:59:41.021533873,1656028781020000000,ASK,1143.62,0.174,5.945419e+06,4873743,5.945387e+06,31.994


In [10]:
lobs = pd.read_csv(path_to_file + 'lobs.csv')

In [11]:
lobs = lobs.rename(columns={
    ' exchange_ts': 'exchange_ts'
})
lobs

,receive_ts,exchange_ts,ethusdt:Binance:LinearPerpetual_ask_price_0,ethusdt:Binance:LinearPerpetual_ask_vol_0,ethusdt:Binance:LinearPerpetual_bid_price_0,ethusdt:Binance:LinearPerpetual_bid_vol_0,ethusdt:Binance:LinearPerpetual_ask_price_1,ethusdt:Binance:LinearPerpetual_ask_vol_1,ethusdt:Binance:LinearPerpetual_bid_price_1,ethusdt:Binance:LinearPerpetual_bid_vol_1,...,ethusdt:Binance:LinearPerpetual_bid_price_7,ethusdt:Binance:LinearPerpetual_bid_vol_7,ethusdt:Binance:LinearPerpetual_ask_price_8,ethusdt:Binance:LinearPerpetual_ask_vol_8,ethusdt:Binance:LinearPerpetual_bid_price_8,ethusdt:Binance:LinearPerpetual_bid_vol_8,ethusdt:Binance:LinearPerpetual_ask_price_9,ethusdt:Binance:LinearPerpetual_ask_vol_9,ethusdt:Binance:LinearPerpetual_bid_price_9,ethusdt:Binance:LinearPerpetual_bid_vol_9
0,1655942402250395360,1655942402249000000,1049.45,1.046,1049.44,0.782,1049.46,2.229,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
1,1655942402656012600,1655942402655000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
2,1655942403348658915,1655942403348000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
3,1655942404080743828,1655942404080000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
4,1655942404151722232,1655942404151000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539694,1656028781406860422,1656028781406000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539695,1656028781441496250,1656028781441000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539696,1656028781474705210,1656028781474000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539697,1656028781508489746,1656028781508000000,1143.63,20.026,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505


In [12]:
lobs.columns

Index(['receive_ts', 'exchange_ts',
       'ethusdt:Binance:LinearPerpetual_ask_price_0',
       'ethusdt:Binance:LinearPerpetual_ask_vol_0',
       'ethusdt:Binance:LinearPerpetual_bid_price_0',
       'ethusdt:Binance:LinearPerpetual_bid_vol_0',
       'ethusdt:Binance:LinearPerpetual_ask_price_1',
       'ethusdt:Binance:LinearPerpetual_ask_vol_1',
       'ethusdt:Binance:LinearPerpetual_bid_price_1',
       'ethusdt:Binance:LinearPerpetual_bid_vol_1',
       'ethusdt:Binance:LinearPerpetual_ask_price_2',
       'ethusdt:Binance:LinearPerpetual_ask_vol_2',
       'ethusdt:Binance:LinearPerpetual_bid_price_2',
       'ethusdt:Binance:LinearPerpetual_bid_vol_2',
       'ethusdt:Binance:LinearPerpetual_ask_price_3',
       'ethusdt:Binance:LinearPerpetual_ask_vol_3',
       'ethusdt:Binance:LinearPerpetual_bid_price_3',
       'ethusdt:Binance:LinearPerpetual_bid_vol_3',
       'ethusdt:Binance:LinearPerpetual_ask_price_4',
       'ethusdt:Binance:LinearPerpetual_ask_vol_4',
       'et

In [12]:
features = lobs.columns

In [13]:
lobs.index = list(lobs.index)
lobs

,receive_ts,exchange_ts,ethusdt:Binance:LinearPerpetual_ask_price_0,ethusdt:Binance:LinearPerpetual_ask_vol_0,ethusdt:Binance:LinearPerpetual_bid_price_0,ethusdt:Binance:LinearPerpetual_bid_vol_0,ethusdt:Binance:LinearPerpetual_ask_price_1,ethusdt:Binance:LinearPerpetual_ask_vol_1,ethusdt:Binance:LinearPerpetual_bid_price_1,ethusdt:Binance:LinearPerpetual_bid_vol_1,...,ethusdt:Binance:LinearPerpetual_bid_price_7,ethusdt:Binance:LinearPerpetual_bid_vol_7,ethusdt:Binance:LinearPerpetual_ask_price_8,ethusdt:Binance:LinearPerpetual_ask_vol_8,ethusdt:Binance:LinearPerpetual_bid_price_8,ethusdt:Binance:LinearPerpetual_bid_vol_8,ethusdt:Binance:LinearPerpetual_ask_price_9,ethusdt:Binance:LinearPerpetual_ask_vol_9,ethusdt:Binance:LinearPerpetual_bid_price_9,ethusdt:Binance:LinearPerpetual_bid_vol_9
0,1655942402250395360,1655942402249000000,1049.45,1.046,1049.44,0.782,1049.46,2.229,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
1,1655942402656012600,1655942402655000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
2,1655942403348658915,1655942403348000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
3,1655942404080743828,1655942404080000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
4,1655942404151722232,1655942404151000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539694,1656028781406860422,1656028781406000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539695,1656028781441496250,1656028781441000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539696,1656028781474705210,1656028781474000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539697,1656028781508489746,1656028781508000000,1143.63,20.026,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505


In [14]:
lobs['receive_ts'] = pd.to_datetime(lobs['receive_ts'])
lobs

,receive_ts,exchange_ts,ethusdt:Binance:LinearPerpetual_ask_price_0,ethusdt:Binance:LinearPerpetual_ask_vol_0,ethusdt:Binance:LinearPerpetual_bid_price_0,ethusdt:Binance:LinearPerpetual_bid_vol_0,ethusdt:Binance:LinearPerpetual_ask_price_1,ethusdt:Binance:LinearPerpetual_ask_vol_1,ethusdt:Binance:LinearPerpetual_bid_price_1,ethusdt:Binance:LinearPerpetual_bid_vol_1,...,ethusdt:Binance:LinearPerpetual_bid_price_7,ethusdt:Binance:LinearPerpetual_bid_vol_7,ethusdt:Binance:LinearPerpetual_ask_price_8,ethusdt:Binance:LinearPerpetual_ask_vol_8,ethusdt:Binance:LinearPerpetual_bid_price_8,ethusdt:Binance:LinearPerpetual_bid_vol_8,ethusdt:Binance:LinearPerpetual_ask_price_9,ethusdt:Binance:LinearPerpetual_ask_vol_9,ethusdt:Binance:LinearPerpetual_bid_price_9,ethusdt:Binance:LinearPerpetual_bid_vol_9
0,2022-06-23 00:00:02.250395360,1655942402249000000,1049.45,1.046,1049.44,0.782,1049.46,2.229,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
1,2022-06-23 00:00:02.656012600,1655942402655000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
2,2022-06-23 00:00:03.348658915,1655942403348000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
3,2022-06-23 00:00:04.080743828,1655942404080000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
4,2022-06-23 00:00:04.151722232,1655942404151000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.31,7.495,1049.53,2.229,1049.30,1.452,1049.54,27.741,1049.29,2.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539694,2022-06-23 23:59:41.406860422,1656028781406000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539695,2022-06-23 23:59:41.441496250,1656028781441000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539696,2022-06-23 23:59:41.474705210,1656028781474000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505
2539697,2022-06-23 23:59:41.508489746,1656028781508000000,1143.63,20.026,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.50,0.016,1143.78,4.445,1143.49,1.755,1143.79,14.975,1143.47,0.505


In [15]:
lobs['supporting: 1_sec_before'] = lobs['receive_ts'].searchsorted(lobs['receive_ts'] - datetime.timedelta(seconds=1))
lobs['supporting: 1_sec_after'] = lobs['receive_ts'].searchsorted(lobs['receive_ts'] + datetime.timedelta(seconds=1))
lobs['supporting: 500_ms_after'] = lobs['receive_ts'].searchsorted(lobs['receive_ts'] + datetime.timedelta(milliseconds=300))

In [16]:
lobs['midprice'] = (lobs['ethusdt:Binance:LinearPerpetual_ask_price_0'] + lobs['ethusdt:Binance:LinearPerpetual_bid_price_0']) / 2
lobs['future: midprice'] = lobs['supporting: 500_ms_after'].map(dict(zip(lobs.index, lobs['midprice'])))

In [17]:
lobs['future: return'] = np.log(lobs['future: midprice']/lobs['midprice'])
lobs

,receive_ts,exchange_ts,ethusdt:Binance:LinearPerpetual_ask_price_0,ethusdt:Binance:LinearPerpetual_ask_vol_0,ethusdt:Binance:LinearPerpetual_bid_price_0,ethusdt:Binance:LinearPerpetual_bid_vol_0,ethusdt:Binance:LinearPerpetual_ask_price_1,ethusdt:Binance:LinearPerpetual_ask_vol_1,ethusdt:Binance:LinearPerpetual_bid_price_1,ethusdt:Binance:LinearPerpetual_bid_vol_1,...,ethusdt:Binance:LinearPerpetual_ask_price_9,ethusdt:Binance:LinearPerpetual_ask_vol_9,ethusdt:Binance:LinearPerpetual_bid_price_9,ethusdt:Binance:LinearPerpetual_bid_vol_9,supporting: 1_sec_before,supporting: 1_sec_after,supporting: 500_ms_after,midprice,future: midprice,future: return
0,2022-06-23 00:00:02.250395360,1655942402249000000,1049.45,1.046,1049.44,0.782,1049.46,2.229,1049.43,5.00,...,1049.54,27.741,1049.29,2.387,0,2,1,1049.445,1049.445,0.0
1,2022-06-23 00:00:02.656012600,1655942402655000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.54,27.741,1049.29,2.387,0,3,2,1049.445,1049.445,0.0
2,2022-06-23 00:00:03.348658915,1655942403348000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.54,27.741,1049.29,2.387,1,5,3,1049.445,1049.445,0.0
3,2022-06-23 00:00:04.080743828,1655942404080000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.54,27.741,1049.29,2.387,2,7,5,1049.445,1049.445,0.0
4,2022-06-23 00:00:04.151722232,1655942404151000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.54,27.741,1049.29,2.387,2,7,5,1049.445,1049.445,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539694,2022-06-23 23:59:41.406860422,1656028781406000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.79,14.975,1143.47,0.505,2539665,2539699,2539699,1143.625,NaN,NaN
2539695,2022-06-23 23:59:41.441496250,1656028781441000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.79,14.975,1143.47,0.505,2539666,2539699,2539699,1143.625,NaN,NaN
2539696,2022-06-23 23:59:41.474705210,1656028781474000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.79,14.975,1143.47,0.505,2539667,2539699,2539699,1143.625,NaN,NaN
2539697,2022-06-23 23:59:41.508489746,1656028781508000000,1143.63,20.026,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.79,14.975,1143.47,0.505,2539668,2539699,2539699,1143.625,NaN,NaN


In [18]:
lobs['i'] = lobs.index

In [19]:
lobs['volatility'] = lobs.progress_apply(lambda x : lobs['future: return'].iloc[x['supporting: 1_sec_before'] : x['i'] + 1].std(), axis=1)
lobs

100%|██████████| 2539699/2539699 [10:49<00:00, 3911.77it/s]


,receive_ts,exchange_ts,ethusdt:Binance:LinearPerpetual_ask_price_0,ethusdt:Binance:LinearPerpetual_ask_vol_0,ethusdt:Binance:LinearPerpetual_bid_price_0,ethusdt:Binance:LinearPerpetual_bid_vol_0,ethusdt:Binance:LinearPerpetual_ask_price_1,ethusdt:Binance:LinearPerpetual_ask_vol_1,ethusdt:Binance:LinearPerpetual_bid_price_1,ethusdt:Binance:LinearPerpetual_bid_vol_1,...,ethusdt:Binance:LinearPerpetual_bid_price_9,ethusdt:Binance:LinearPerpetual_bid_vol_9,supporting: 1_sec_before,supporting: 1_sec_after,supporting: 500_ms_after,midprice,future: midprice,future: return,i,volatility
0,2022-06-23 00:00:02.250395360,1655942402249000000,1049.45,1.046,1049.44,0.782,1049.46,2.229,1049.43,5.00,...,1049.29,2.387,0,2,1,1049.445,1049.445,0.0,0,NaN
1,2022-06-23 00:00:02.656012600,1655942402655000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.29,2.387,0,3,2,1049.445,1049.445,0.0,1,0.000000
2,2022-06-23 00:00:03.348658915,1655942403348000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.29,2.387,1,5,3,1049.445,1049.445,0.0,2,0.000000
3,2022-06-23 00:00:04.080743828,1655942404080000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.29,2.387,2,7,5,1049.445,1049.445,0.0,3,0.000000
4,2022-06-23 00:00:04.151722232,1655942404151000000,1049.45,0.314,1049.44,19.860,1049.46,2.243,1049.43,5.00,...,1049.29,2.387,2,7,5,1049.445,1049.445,0.0,4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539694,2022-06-23 23:59:41.406860422,1656028781406000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.47,0.505,2539665,2539699,2539699,1143.625,NaN,NaN,2539694,0.000037
2539695,2022-06-23 23:59:41.441496250,1656028781441000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.47,0.505,2539666,2539699,2539699,1143.625,NaN,NaN,2539695,0.000036
2539696,2022-06-23 23:59:41.474705210,1656028781474000000,1143.63,19.992,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.47,0.505,2539667,2539699,2539699,1143.625,NaN,NaN,2539696,0.000035
2539697,2022-06-23 23:59:41.508489746,1656028781508000000,1143.63,20.026,1143.62,23.868,1143.67,2.348,1143.61,4.35,...,1143.47,0.505,2539668,2539699,2539699,1143.625,NaN,NaN,2539697,0.000034


In [20]:
lobs['future: volatility'] = lobs['supporting: 1_sec_after'].map(dict(zip(lobs.index, lobs['volatility'])))

In [32]:
lobs['order_intensity'] = trades['order_intensity'][trades['receive_ts'].searchsorted(lobs['receive_ts']).clip(0, trades.shape[0] - 1)].reset_index(drop=True)

lobs['future: order_intensity'] = trades['order_intensity'][trades['receive_ts'].searchsorted(lobs['receive_ts'] + datetime.timedelta(seconds=2)).clip(0, trades.shape[0] - 1)].reset_index(drop=True)

In [33]:
lobs['order_intensity'][2000000:]

2000000    36.462
2000001    36.462
2000002    36.462
2000003    36.674
2000004    36.958
            ...  
2539694    32.994
2539695    32.994
2539696    32.994
2539697    32.994
2539698    32.994
Name: order_intensity, Length: 539699, dtype: float64

In [34]:
lobs = lobs.iloc[1000:-1000]

In [35]:
targets = {
    'return': lobs['future: return'].copy(deep=True),
    'volatility': lobs['future: volatility'].copy(deep=True),
    'order_intensity': lobs['future: order_intensity'].copy(deep=True)
}

In [36]:
lobs = lobs[features]

## Predict

In [43]:
from lightgbm import LGBMRegressor
import joblib

In [38]:
import re
lobs = lobs.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '-', x))
lobs

,receive_ts,exchange_ts,ethusdt-Binance-LinearPerpetual_ask_price_0,ethusdt-Binance-LinearPerpetual_ask_vol_0,ethusdt-Binance-LinearPerpetual_bid_price_0,ethusdt-Binance-LinearPerpetual_bid_vol_0,ethusdt-Binance-LinearPerpetual_ask_price_1,ethusdt-Binance-LinearPerpetual_ask_vol_1,ethusdt-Binance-LinearPerpetual_bid_price_1,ethusdt-Binance-LinearPerpetual_bid_vol_1,...,ethusdt-Binance-LinearPerpetual_bid_price_7,ethusdt-Binance-LinearPerpetual_bid_vol_7,ethusdt-Binance-LinearPerpetual_ask_price_8,ethusdt-Binance-LinearPerpetual_ask_vol_8,ethusdt-Binance-LinearPerpetual_bid_price_8,ethusdt-Binance-LinearPerpetual_bid_vol_8,ethusdt-Binance-LinearPerpetual_ask_price_9,ethusdt-Binance-LinearPerpetual_ask_vol_9,ethusdt-Binance-LinearPerpetual_bid_price_9,ethusdt-Binance-LinearPerpetual_bid_vol_9
1000,2022-06-23 00:00:35.821945636,1655942435820000000,1050.12,9.603,1050.11,0.771,1050.13,15.593,1050.10,0.600,...,1050.04,2.327,1050.20,2.120,1050.03,4.095,1050.21,7.110,1050.02,11.000
1001,2022-06-23 00:00:35.854527140,1655942435853000000,1050.12,10.190,1050.09,0.019,1050.13,15.412,1050.07,4.483,...,1050.00,0.006,1050.20,2.120,1049.99,0.072,1050.21,7.110,1049.98,11.600
1002,2022-06-23 00:00:35.886630080,1655942435885000000,1050.00,0.037,1049.97,0.234,1050.10,8.625,1049.96,0.133,...,1049.81,1.800,1050.18,1.910,1049.80,2.020,1050.19,11.910,1049.79,1.800
1003,2022-06-23 00:00:35.919177172,1655942435917000000,1049.98,1.300,1049.97,0.209,1050.00,0.037,1049.96,0.133,...,1049.81,1.800,1050.15,1.910,1049.80,2.020,1050.16,38.225,1049.79,1.800
1004,2022-06-23 00:00:35.952282357,1655942435950000000,1049.98,1.295,1049.97,0.194,1050.00,0.037,1049.96,0.133,...,1049.82,0.600,1050.12,114.120,1049.81,1.800,1050.13,113.917,1049.80,2.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538694,2022-06-23 23:59:07.764691524,1656028747764000000,1143.17,124.950,1143.16,1.332,1143.18,41.895,1143.15,2.136,...,1143.02,0.006,1143.28,8.094,1143.01,3.194,1143.29,9.190,1143.00,7.491
2538695,2022-06-23 23:59:07.798780050,1656028747798000000,1143.17,124.950,1143.16,1.337,1143.18,40.848,1143.15,2.136,...,1143.01,3.194,1143.27,8.467,1143.00,7.491,1143.28,8.094,1142.99,0.492
2538696,2022-06-23 23:59:07.832533055,1656028747832000000,1143.16,0.848,1143.15,2.136,1143.17,140.813,1143.11,0.013,...,1143.00,7.491,1143.25,1.047,1142.99,0.492,1143.26,4.814,1142.98,4.350
2538697,2022-06-23 23:59:07.865833387,1656028747865000000,1143.12,0.600,1143.11,0.013,1143.16,149.650,1143.10,0.456,...,1142.99,0.492,1143.26,8.414,1142.98,4.350,1143.27,4.618,1142.92,1.740


In [44]:
params = {'objective': 'regression', 'learning_rate': 0.1, 'num_leaves': 8, 'early_stopping_round': 10,
          'verbose': -1, 'linear_tree': True}

model = LGBMRegressor(**params)
X_train = lobs[:1800000].drop(columns=['receive_ts', 'exchange_ts'])
y_train = targets['return'][:1800000]

X_es = lobs[1800000:2100000].drop(columns=['receive_ts', 'exchange_ts'])
y_es = targets['return'][1800000:2100000]

X_val = lobs[2100000:].drop(columns=['receive_ts', 'exchange_ts'])
y_val = targets['return'][2100000:]

model.fit(X_train, y_train, eval_set=[(X_es, y_es)])

joblib.dump(model, 'return_predictor.pkl')

preds = model.predict(X_val)

np.corrcoef(y_val, preds)

[1]	valid_0's l2: 8.23368e-09
[2]	valid_0's l2: 8.14821e-09
[3]	valid_0's l2: 8.07784e-09
[4]	valid_0's l2: 8.01851e-09
[5]	valid_0's l2: 7.96993e-09
[6]	valid_0's l2: 7.93e-09
[7]	valid_0's l2: 7.8984e-09
[8]	valid_0's l2: 7.87093e-09
[9]	valid_0's l2: 7.8461e-09
[10]	valid_0's l2: 7.82585e-09
[11]	valid_0's l2: 7.80858e-09
[12]	valid_0's l2: 7.79362e-09
[13]	valid_0's l2: 7.77982e-09
[14]	valid_0's l2: 7.76736e-09
[15]	valid_0's l2: 7.75727e-09
[16]	valid_0's l2: 7.74697e-09
[17]	valid_0's l2: 7.73893e-09
[18]	valid_0's l2: 7.73076e-09
[19]	valid_0's l2: 7.72388e-09
[20]	valid_0's l2: 7.71716e-09
[21]	valid_0's l2: 7.71123e-09
[22]	valid_0's l2: 7.7057e-09
[23]	valid_0's l2: 7.70175e-09
[24]	valid_0's l2: 7.69687e-09
[25]	valid_0's l2: 7.69283e-09
[26]	valid_0's l2: 7.68923e-09
[27]	valid_0's l2: 7.68592e-09
[28]	valid_0's l2: 7.68266e-09
[29]	valid_0's l2: 7.67953e-09
[30]	valid_0's l2: 7.67687e-09
[31]	valid_0's l2: 7.67419e-09
[32]	valid_0's l2: 7.67199e-09
[33]	valid_0's l2: 7.67

array([[1.        , 0.15603302],
       [0.15603302, 1.        ]])

In [45]:
params = {'objective': 'regression', 'learning_rate': 0.1, 'num_leaves': 8, 'early_stopping_round': 10,
          'verbose': -1, 'linear_tree': True}

model = LGBMRegressor(**params)
X_train = lobs[:1800000].drop(columns=['receive_ts', 'exchange_ts'])
y_train = targets['volatility'][:1800000]

X_es = lobs[1800000:2100000].drop(columns=['receive_ts', 'exchange_ts'])
y_es = targets['volatility'][1800000:2100000]

X_val = lobs[2100000:].drop(columns=['receive_ts', 'exchange_ts'])
y_val = targets['volatility'][2100000:]

model.fit(X_train, y_train, eval_set=[(X_es, y_es)])

joblib.dump(model, 'volatility_predictor.pkl')

preds = model.predict(X_val)

np.corrcoef(y_val, preds)

[1]	valid_0's l2: 3.38414e-09
[2]	valid_0's l2: 3.36933e-09
[3]	valid_0's l2: 3.34597e-09
[4]	valid_0's l2: 3.3348e-09
[5]	valid_0's l2: 3.32837e-09
[6]	valid_0's l2: 3.31379e-09
[7]	valid_0's l2: 3.30988e-09
[8]	valid_0's l2: 3.3067e-09
[9]	valid_0's l2: 3.29442e-09
[10]	valid_0's l2: 3.29257e-09
[11]	valid_0's l2: 3.28603e-09
[12]	valid_0's l2: 3.28403e-09
[13]	valid_0's l2: 3.28354e-09
[14]	valid_0's l2: 3.27934e-09
[15]	valid_0's l2: 3.27534e-09
[16]	valid_0's l2: 3.27211e-09
[17]	valid_0's l2: 3.27036e-09
[18]	valid_0's l2: 3.26928e-09
[19]	valid_0's l2: 3.26545e-09
[20]	valid_0's l2: 3.26198e-09
[21]	valid_0's l2: 3.26054e-09
[22]	valid_0's l2: 3.26079e-09
[23]	valid_0's l2: 3.25951e-09
[24]	valid_0's l2: 3.26081e-09
[25]	valid_0's l2: 3.26184e-09
[26]	valid_0's l2: 3.2616e-09
[27]	valid_0's l2: 3.25631e-09
[28]	valid_0's l2: 3.25557e-09
[29]	valid_0's l2: 3.25534e-09
[30]	valid_0's l2: 3.25471e-09
[31]	valid_0's l2: 3.25494e-09
[32]	valid_0's l2: 3.25473e-09
[33]	valid_0's l2: 3

array([[1.        , 0.19519253],
       [0.19519253, 1.        ]])

In [46]:
params = {'objective': 'regression', 'learning_rate': 0.1, 'num_leaves': 8, 'early_stopping_round': 10,
          'verbose': -1, 'linear_tree': True}

model = LGBMRegressor()
X_train = lobs[:1800000].drop(columns=['receive_ts', 'exchange_ts'])
y_train = targets['order_intensity'][:1800000]

X_es = lobs[1800000:2100000].drop(columns=['receive_ts', 'exchange_ts'])
y_es = targets['order_intensity'][1800000:2100000]

X_val = lobs[2100000:].drop(columns=['receive_ts', 'exchange_ts'])
y_val = targets['order_intensity'][2100000:]

model.fit(X_train, y_train)

joblib.dump(model, 'order_intensity_predictor.pkl')

preds = model.predict(X_val)

np.corrcoef(y_val, preds)

array([[1.        , 0.05809047],
       [0.05809047, 1.        ]])

Запустить предсказательные модели на последнем ордербуке и юзать их как вместо текущих значений

In [ ]:
y_val

Первые 12 часов как трейн
Вторые как valid
Предсказываю midprice - order_intensity - volatility

In [ ]:
reload(simulator)
reload(Stoikov)
reload(get_info)
reload(load_data)

In [ ]:
md = load_data.load_md_from_file(path=path_to_file, run_time=run_time)

In [ ]:
latency = pd.Timedelta(10, 'ms').delta
md_latency = pd.Timedelta(10, 'ms').delta

sim = simulator.SimOptim(md, latency, md_latency)

In [ ]:
#delay between orders
delay = pd.Timedelta(0.01, 's').delta

strategy = Stoikov.Strategy(
    delay=delay,
    risk_koef=1,
    time_oi=60*10**9,
    avg_sum_oi=800,
    avg_time_oi=57471412238,
    order_fees=0.00001,
    avg_volatility=1e-5,
    min_asset_value=0.001,
    volatility_record_cooldown=0.1*10**9, #ns
    volatility_horizon=1000,
    order_intensity_min_samples=100
)
# вся эта история с avg нужна для того, чтобы order_intensity и volatility были равны 1 в среднем.

In [ ]:
trades_list, md_list, updates_list, all_orders = strategy.run(sim)

In [ ]:
logs = strategy.logs

In [ ]:
to_save = pd.DataFrame({
    'time': logs['place_order_time'],
    'midprice': logs['midprice'],
    'volatility': logs['volatility'],
    'order_intensity': logs['order_intensity']
})

In [ ]:
to_save.to_csv('12h_mm_data')

In [ ]:
plt.plot(pd.to_datetime(logs['place_order_time']), logs['volatility'])

In [ ]:
plt.plot(pd.to_datetime(logs['own_trade_time']), logs['asset_position'])

In [ ]:
len(logs['pnl_with_liq']) # 57972

In [ ]:
print(1/0)

In [ ]:
trades_list, md_list, updates_list, all_orders = strategy.run(sim)

In [ ]:
logs = strategy.logs

In [ ]:
plt.plot(logs['midprice'])

In [ ]:
plt.plot(logs['volatility'])

In [ ]:
pd.to_datetime(logs['place_order_time'])